In [1]:
#import library
from random import randint
import random
import numpy as np
import pandas as pd
from openpyxl import load_workbook

#import dataset
wb = load_workbook(filename= 'Dataset.xlsx')
sheet_ranges = wb['Sheet 1']

df = pd.DataFrame(sheet_ranges.values)
df

#menampilkan data yang dibutuhkan
col = df[1:72][[4,5,6,7,9,10,11,12,13]]
col.columns = ['UAW','S','A','C','UUCW','TCF','ECF','Real','Real_Effort']
col1 = pd.DataFrame(col)

col.to_excel('col1.xlsx')

for index, row in col1.iterrows():
    UAW = col1.loc[index,"UAW"]
    UUCW = col1.loc[index,"UUCW"]
    TCF = col1.loc[index,"TCF"]
    ECF = col1.loc[index,"ECF"]
    Real_E = col1.loc[index,"Real_Effort"]
    
    #desired_UUCW = Real_E/((TCF*ECF)*20)-UAW
    #col1.at(5,"UUCW PSO", desired_UUCW , True)
    #col1.at[index, 'Desired_UUCW'] = np.round(desired_UUCW,1)


col1

,UAW,S,A,C,UUCW,TCF,ECF,Real,Real_Effort
1,9,6,10,15,355,0.81,0.84,398.5,7970
2,8,4,20,15,445,0.99,0.99,398.1,7962
3,9,1,5,20,355,1.03,0.8,396.75,7935
4,8,5,10,15,350,0.9,0.91,390.25,7805
5,8,1,10,16,345,0.9,0.91,387.9,7758
...,...,...,...,...,...,...,...,...,...
67,18,2,18,18,460,1.05,0.95,290,5800
68,17,1,23,22,565,1.03,0.8,289.55,5791
69,19,5,30,10,475,0.95,0.92,289.1,5782
70,6,5,15,5,250,1,0.92,288.9,5778


In [3]:
all_results = []
loop = 1000 #Perulangan keseluruhan program
totaldata = 71 #Total data yang diproses
mean_sac = [] #Menyimpan posisi akhir

#Ulang proses 1000 kali
for i in range(loop):
    #print('try ', i)
    mae = 0
    final_results = []
    for index, row in col1.iterrows():
        #print()
        #print("____ DATA ",index,"____")

        #deklarasi variabel pso
        swarm_size = 70
        pf = 20
        max_iter = 50
        c1 = 2
        c2 = 2
        posisi = {}
        partikel = {}
        fitness_value = 200

        #deklarasi batas atas dan bawah
        rentangposisi = {
            "simple": {
                "lower": 5,
                "upper": 7.49
            },
            "average": {
                "lower": 7.5,
                "upper": 12.49
            },
            "complex": {
                "lower": 12.5,
                "upper": 15
            }
        }

        #deklarasi variabel UCP
        projectactual = {
            'simpleUC': col1.loc[index,"S"],
            'averageUC': col1.loc[index,"A"],
            'complexUC': col1.loc[index,"C"],
            'uaw': col1.loc[index,"UAW"],
            'tcf': col1.loc[index,"TCF"],
            'ecf': col1.loc[index,"ECF"],
            'actualEffort': col1.loc[index,"Real_Effort"]
        }


        #fungsi generate posisi acak
        def posisiacak():
            xsimple = random.uniform(rentangposisi['simple']['lower'], rentangposisi['simple']['upper'])
            xaverage = random.uniform(rentangposisi['average']['lower'], rentangposisi['average']['upper'])
            xcomplex = random.uniform(rentangposisi['complex']['lower'], rentangposisi['complex']['upper'])
            position = {
                        'simple':xsimple,
                        'average':xaverage,
                        'complex':xcomplex
                        }   
            return position
        
        #fungsi hitung UCP
        def size(xsimple, xaverage, xcomplex):
            ucsimple = xsimple * projectactual['simpleUC']
            ucaverage = xaverage * projectactual['averageUC']
            uccomplex = xcomplex * projectactual['complexUC']
            uucw = ucsimple + ucaverage + uccomplex
            uucp = projectactual['uaw'] + uucw
            return uucp * projectactual['tcf'] * projectactual['ecf']

        #fungsi update velocity
        def velocity(parameters):
            vsimple = (parameters['w'] * parameters['partikel']['vsimple']) + ((c1 * parameters['r1']) * (parameters['pbest']['simple'] - parameters['partikel']['simple'])) + ((c2 * parameters['r2']) * (parameters['gbest']['simple'] - parameters['partikel']['simple']))
            vaverage = (parameters['w'] * parameters['partikel']['vaverage']) + ((c1 * parameters['r1']) * (parameters['pbest']['average'] - parameters['partikel']['average'])) + ((c2 * parameters['r2']) * (parameters['gbest']['average'] - parameters['partikel']['average']))
            vcomplex = (parameters['w'] * parameters['partikel']['vcomplex']) + ((c1 * parameters['r1']) * (parameters['pbest']['complex'] - parameters['partikel']['complex'])) + ((c2 * parameters['r2']) * (parameters['gbest']['complex'] - parameters['partikel']['complex']))
            return {'vsimple': vsimple, 'vaverage': vaverage, 'vcomplex': vcomplex}

        #fungsi perubahan inertia
        def inertia(iterasi):
            inertia_max = 0.9
            inertia_min = 0.4
            return inertia_min - (((inertia_max - inertia_min) * iterasi) / max_iter)

        #fungsi pembatas atas & bawah
        def exceedlimit(value, label):
            if label == 'simple' and value < rentangposisi['simple']['lower']:
                value = rentangposisi['simple']['lower']
            if label == 'simple' and value > rentangposisi['simple']['upper']:
                value = rentangposisi['simple']['upper']
            if label == 'average' and value < rentangposisi['average']['lower']:
                value = rentangposisi['average']['lower']
            if label == 'average' and value > rentangposisi['average']['upper']:
                value = rentangposisi['average']['upper']
            if label == 'complex' and value < rentangposisi['complex']['lower']:
                value = rentangposisi['complex']['lower']
            if label == 'complex' and value > rentangposisi['complex']['upper']:
                value = rentangposisi['complex']['upper']
            return value

        #fungsi utama
        def optimizer():
            partikel = []
            gbests = []
            pbests = []
            ret = []
            for iter in range(max_iter):
                w = inertia(iter) #Generate Inertia
                r1 = random.uniform(0, 1) #Generate r1
                r2 = random.uniform(0, 1) #Generate r2

                # initial population
                if iter == 0: 
                    for i in range(swarm_size):
                        #Inisialisasi posisi awal
                        xsimple = posisiacak()['simple']
                        xaverage = posisiacak()['average']
                        xcomplex = posisiacak()['complex']

                        #Hitung nilai effort
                        software_size = size(xsimple,xaverage,xcomplex)
                        estimated_effort = software_size * pf

                        ae = abs(projectactual['actualEffort'] - estimated_effort) #Hitung selisih effort aktual dan estimasi

                        #Inisialisasi velocity awal
                        vsimple = random.uniform(0, 1)
                        vaverage = random.uniform(0, 1)
                        vcomplex = random.uniform(0, 1)

                        #Simpan partikel
                        particles = {
                            'simple': xsimple,
                            'average': xaverage,
                            'complex': xcomplex,
                            'size': software_size,
                            'estimated': estimated_effort,
                            'ae': ae,
                            'vsimple':vsimple,
                            'vaverage':vaverage,
                            'vcomplex':vcomplex
                        }
                        partikel.append([])
                        partikel[iter].append(particles)

                        #Simpan PBest
                        pbests.append([])
                        pbests[iter].append(particles)
                    min_ae = min(x['ae'] for x in pbests[iter]) #Mencari PBest dengan ae terkecil

                    #Menentukan dan simpan GBest
                    gbest = next(item for item in pbests[iter] if item['ae'] == min_ae)
                    gbests.append([])
                    gbests[iter].append(gbest)

                if iter > 0:
                    for count,value in enumerate(partikel[iter-1]):
                        parameters = {
                            'pbest': pbests[iter-1][count],
                            'partikel': value,
                            'gbest': gbests[iter-1][0],
                            'w':w,
                            'r1':r1,
                            'r2':r2
                        }
   
                        vel = velocity(parameters) #Update velocity

                        #Update posisi
                        xsimple = value['simple'] + vel['vsimple']
                        xaverage = value['average'] + vel['vaverage']
                        xcomplex = value['complex'] + vel['vcomplex']

                        #Exceedlimit
                        xsimple = exceedlimit(xsimple, 'simple')
                        xaverage = exceedlimit(xaverage, 'average')
                        xcomplex = exceedlimit(xcomplex, 'complex')

                        #Hitung nilai effort
                        software_size = size(xsimple,xaverage,xcomplex)
                        estimated_effort = software_size * pf

                        ae = abs(projectactual['actualEffort'] - estimated_effort) #Hitung selisih effort aktual dan estimasi

                        #Simpan partikel
                        particles = {
                            'simple': xsimple,
                            'average': xaverage,
                            'complex': xcomplex,
                            'size': software_size,
                            'estimated': estimated_effort,
                            'ae': ae,
                            'vsimple':vel['vsimple'],
                            'vaverage':vel['vaverage'],
                            'vcomplex':vel['vcomplex']
                        }
                        partikel.append([])
                        partikel[iter].append(particles)
                        
                        #Menentukan PBest
                        if pbests[iter-1][count]['ae'] > partikel[iter][count]['ae']:
                           pbests[iter].append(partikel[iter][count])
                        else:
                            pbests[iter].append(partikel[iter-1][count])

                    #Menentukan GBest
                    min_ae = min(x['ae'] for x in pbests[iter])
                    gbest = next(item for item in pbests[iter] if item['ae'] == min_ae)
                    gbests.append([])
                    gbests[iter].append(gbest)
                    if gbests[iter][0]['ae'] < fitness_value:
                        return gbests[iter][0]
                    ret.append(gbests[iter][0])
            return ret

        results = optimizer()

        if len(results) == 9:
            mae = mae + results['ae']
            final_results.append(results)
                    
        else:
            min_ae = min(x['ae'] for x in results)
            results = next(item for item in results if item['ae'] == min_ae)
            mae = mae + results['ae']
            final_results.append(results)
        
    all_results.append(final_results)
   
#Menghitung rata-rata dari 1000 loop
for i in range(totaldata): 
    total_simple = 0 
    total_average = 0 
    total_complex = 0 
    for x in (all_results): 
        total_simple = total_simple + x[i]['simple'] 
        total_average = total_average + x[i]['average'] 
        total_complex = total_complex + x[i]['complex'] 

    mean_simple = total_simple/loop 
    mean_average = total_average/loop 
    mean_complex = total_complex/loop 

    mean_sac.append([mean_simple, mean_average, mean_complex])
    print('Mean xsimple, mean xaverage, mean xcomplex dataset ke-', i, '=', mean_sac[i])

print('success')

Mean xsimple, mean xaverage, mean xcomplex dataset ke- 0 = [7.47820612528625, 12.48999999999982, 15.0]
Mean xsimple, mean xaverage, mean xcomplex dataset ke- 1 = [6.296165066343276, 8.332298540458758, 13.758999552361546]
Mean xsimple, mean xaverage, mean xcomplex dataset ke- 2 = [7.4286200273915775, 12.489696918959389, 15.0]
Mean xsimple, mean xaverage, mean xcomplex dataset ke- 3 = [7.476615754839825, 12.48999999999982, 15.0]
Mean xsimple, mean xaverage, mean xcomplex dataset ke- 4 = [7.425004737844375, 12.48999999999982, 15.0]
Mean xsimple, mean xaverage, mean xcomplex dataset ke- 5 = [6.748164526832709, 12.444687716309117, 14.960577214029573]
Mean xsimple, mean xaverage, mean xcomplex dataset ke- 6 = [6.328289775962561, 8.889639018422166, 13.788268540515908]
Mean xsimple, mean xaverage, mean xcomplex dataset ke- 7 = [6.400094878259187, 12.48999999999982, 15.0]
Mean xsimple, mean xaverage, mean xcomplex dataset ke- 8 = [7.449701642817538, 12.48965506356186, 15.0]
Mean xsimple, mean x

In [4]:
#print(mean_sac)
final_ae = []
mae = 0

#mengatur ukuran maksimal tabel
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

#menampilkan nilai yang dibutuhkan
hasil = df[1:72][[5,6,7,13]]
hasil.columns = ['S','A','C','Real_Effort']
hasil1 = pd.DataFrame(hasil)

for index, row in col1.iterrows():
        #print("____ DATA ",index,"____")

        #mengambil nilai gbest terakhir
        xsimple = mean_sac[index-1][0]
        xaverage = mean_sac[index-1][1]
        xcomplex = mean_sac[index-1][2]

        #Menambahkan kolom-kolom gbest
        hasil1.at[index, 'xS'] = np.round(xsimple,1)
        hasil1.at[index, 'xA'] = np.round(xaverage,1)
        hasil1.at[index, 'xC'] = np.round(xcomplex,1)

        projectactual = {
                'simpleUC': col1.loc[index,"S"],
                'averageUC': col1.loc[index,"A"],
                'complexUC': col1.loc[index,"C"],
                'uaw': col1.loc[index,"UAW"],
                'tcf': col1.loc[index,"TCF"],
                'ecf': col1.loc[index,"ECF"],
                'actualEffort': col1.loc[index,"Real_Effort"]
        }

        def size(xsimple, xaverage, xcomplex):
                ucsimple = xsimple * projectactual['simpleUC']
                ucaverage = xaverage * projectactual['averageUC']
                uccomplex = xcomplex * projectactual['complexUC']
                uucw = ucsimple + ucaverage + uccomplex
                uucp = projectactual['uaw'] + uucw
                return uucp * projectactual['tcf'] * projectactual['ecf']

        #Menghitung UCP
        software_size = size(xsimple,xaverage,xcomplex)
        estimated_effort = software_size * pf
        ae = abs(projectactual['actualEffort'] - estimated_effort)

        #Memasukkan ae dalam array
        final_ae.append(ae)

        #Menambahkan kolom estimated effort dan ae
        hasil1.at[index, 'e_effort'] = np.round(estimated_effort,1)
        hasil1.at[index, 'ae'] = np.round(ae,1)


print(hasil1)
print()

hasil.to_excel('hasil1.xlsx') #Simpan Hasil ke Excel

#Menentukan MAE
mae = np.mean(final_ae)
print('mae', mae)


     S   A   C Real_Effort   xS    xA    xC  e_effort      ae
1    6  10  15        7970  7.5  12.5  15.0    5494.5  2475.5
2    4  20  15        7962  6.3   8.3  13.8    7962.6     0.6
3    1   5  20        7935  7.4  12.5  15.0    6243.9  1691.1
4    5  10  15        7805  7.5  12.5  15.0    6474.7  1330.3
5    1  10  16        7758  7.4  12.5  15.0    6229.7  1528.3
6    1  13  14        7643  6.7  12.4  15.0    7565.9    77.1
7    3  18  15        7532  6.3   8.9  13.8    7532.8     0.8
8    0  16  12        7451  6.4  12.5  15.0    6391.6  1059.4
9    2  10  15        7449  7.4  12.5  15.0    7148.7   300.3
10   4  14  17        7427  6.2   8.2  13.1    7429.3     2.3
11   5  16  10        7406  7.5  12.5  15.0    5672.3  1733.7
12   1  10  15        7365  7.4  12.5  15.0    5527.4  1837.6
13   9   8  19        7350  7.5  12.5  15.0    5580.7  1769.3
14   5   8  20        7303  5.1   7.5  12.5    7576.2   273.2
15   2  15  11        7252  6.4  11.7  13.9    7252.6     0.6
16   1  